In [1]:
import os

In [2]:
%pwd

'/home/adhitizki/playground/pacmann/e2e_mlops_credit/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/adhitizki/playground/pacmann/e2e_mlops_credit'

### Training Config

This code will be apply in `src/MLProject/entity/config_entity.py`.

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    input_train_path: Path
    output_train_path: Path
    scaled_train_path: Path
    model_path: Path
    params_max_iter: int
    params_solver: str
    params_n_jobs: int

### Training Config Manager

This code will be apply in `src/MLProject/config/configurations.py`.

In [6]:
from MLProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from MLProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        """read training config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: TrainingConfig type
        """
        data_dump_config = self.config.dump_data
        scaler_config = self.config.scaler_data
        train_config = self.config.train_model
        train_params = self.params

        create_directories([train_config.root_dir])

        config = TrainingConfig(
            root_dir=train_config.root_dir,
            input_train_path=Path(data_dump_config.input_train_path),
            output_train_path=Path(data_dump_config.output_train_path),
            scaled_train_path=Path(scaler_config.scaled_train_path),
            model_path=Path(train_config.model_path),
            params_max_iter=train_params.MAX_ITER,
            params_solver=train_params.SOLVER,
            params_n_jobs=train_params.N_JOBS
        )

        return config

### Perform Training

This code in `src/MLProject/components/training.py`.

For this example, for initial run we could use logistic regression, later on we could try:
+ another model
+ another data enrichment technique
+ another model tweaking

In [8]:
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from MLProject import logger

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def logistic_regression(self) -> None:
        """train the data with linear regression model and dump the data
        """
        logger.info(f"Load scaled data train from {self.config.scaled_train_path}.")
        X_train_scaled = joblib.load(self.config.scaled_train_path)
        
        logger.info(f"Load data train output from {self.config.output_train_path}.")
        y_train = joblib.load(self.config.output_train_path)
        
        logger.info(f"Train the model.")
        model = LogisticRegression(
            solver=self.config.params_solver,
            max_iter=self.config.params_max_iter,
            n_jobs=self.config.params_n_jobs
        )
        
        model.fit(X_train_scaled, y_train)
        
        logger.info(f"Dump the model.")
        joblib.dump(model, self.config.model_path)

### Traning the Model

This code in `src/MLProject/pipeline/step_03_training.py`.

In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.logistic_regression()
except Exception as e:
    logger.error(e)
    raise e

[2024-07-08 23:45:54,909: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-08 23:45:54,913: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-07-08 23:45:54,914: INFO: common: created directory at: artifacts]
[2024-07-08 23:45:54,915: INFO: common: created directory at: artifacts/models]
[2024-07-08 23:45:54,917: INFO: 1447342053: Load scaled data train from artifacts/preprocessing/X_train_scaled.pkl.]
[2024-07-08 23:45:54,937: INFO: 1447342053: Load data train output from artifacts/data/y_train.pkl.]
[2024-07-08 23:45:55,226: INFO: 1447342053: Train the model.]
[2024-07-08 23:45:56,539: INFO: 1447342053: Dump the model.]


**Debug**: Predict by showing the data training prediction result.

In [10]:
import pandas as pd

X_train = joblib.load(training_config.input_train_path)
X_train_scaled = joblib.load(training_config.scaled_train_path)
y_train = joblib.load(training_config.output_train_path)
model = joblib.load(training_config.model_path)

y_pred = pd.Series(model.predict(X_train_scaled), index = X_train.index)
y_pred

325383    1
243312    0
475129    1
291944    1
564915    1
         ..
236610    0
340987    1
556907    1
360063    1
366060    1
Length: 113726, dtype: int64

In [11]:
print(f"{classification_report(y_train, y_pred)}")

              precision    recall  f1-score   support

           0       0.95      0.98      0.97     56863
           1       0.98      0.95      0.96     56863

    accuracy                           0.96    113726
   macro avg       0.97      0.96      0.96    113726
weighted avg       0.97      0.96      0.96    113726

